In [ ]:
# Import packages
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_json("data/structured_data.json", orient="table")
df.head()

# Plotly visualisation

## 1. Bar chart
Bar chart of the number of recipes per category or subcategory

In [ ]:
import plotly.express as px

In [ ]:
# Bar chart using plotly
recipes_count_per_category = df.groupby("category").count()["recipe_name"].reset_index()
recipes_count_per_category.columns = ["category", "count"]
recipes_count_per_category

In [ ]:
fig = px.bar(
    recipes_count_per_category,
    x="category",
    y="count",
    title="Число рецептов по категориям",
)
fig.show()

In [ ]:
# recipes_count_per_subcategory = \
#     df.groupby(["category", "subcategory"]).count()["recipe_name"].reset_index()
# recipes_count_per_subcategory.columns = ["category", "subcategory", "count"]
# recipes_count_per_subcategory

In [ ]:
# fig = px.bar(
#     recipes_count_per_subcategory,
#     x="subcategory",
#     y="count",
#     color="category",
#     title="Number of recipes per subcategory",
# )
# fig.show()

## 2. Word cloud
Word cloud of the most commonly used ingredients

In [ ]:
from collections import defaultdict

In [ ]:
# Word cloud
ingredients_list_counts = defaultdict(int)

for ingredients in df["parsed_ingredients"]:
    for ingredient in ingredients:
        if ingredient:
            ingredients_list_counts[ingredient[0]] += 1

# Save to file
with open("visualisation/data/ingredients.txt", "w", encoding="utf-8") as f:
    for ingredient, count in ingredients_list_counts.items():
        f.write(f"{ingredient} {count}\n")

In [ ]:
# Show svg file
from IPython.display import SVG, display

display(SVG(filename="visualisation/wordcloud.svg"))